#### Física de la Radioterapia. Máster de Física Biomédica. Universidad Complutense de Madrid
# Seminario DICOM. Datos sintéticos

### Instalación de plastimatch en Colab

`plastimatch` es un programa de registro de imágenes en radioterapia con altas capacidades para el manejo de distintos formatos, en concreto de estudios **DICOM**.

**Google Colab** utiliza una distribución Ubuntu sin paquete `plastimatch`. Podemos intentar instalarlo mediante el comando usual:

`apt install plastimatch`

pero en el caso de que los binarios de `plastimatch` no estén disponibles para la versión de Ubuntu el comando `apt` nos devolverá:

`Package 'plastimatch' has no installation candidate`.

Afortunadamente el módulo `pyplastimatch` resuelve esta limitación empequetando su propia compilación.

La siguiente celda instala el ejecutable `plastimatch` y el módulo `pyplastimatch` que lo encapsula.

In [ ]:
!uv pip install pyplastimatch
from pyplastimatch.utils.install import install_precompiled_binaries
install_precompiled_binaries()

## Creación de un maniquí sintético

Corremos una secuencia de comandos de plastimatch
- Creación de las imágenes del paciente, una cuba de agua
- Dos máscaras para la estructuras:
  + BODY: coincide con el contorno externo de la cuba
  + PTV: Una esfera en el centro de la cuba
- Creamos el archivo DICOM de estructuras a partir de las máscaras anteriores

In [ ]:
# Creación de la estructura DICOM de datos. Plastimatch crea los archivos de imagen, un archivo de estructuras y otro de dosis.
!plastimatch synth \
  --background -1000 \
  --foreground 0 \
  --pattern rect \
  --output-dicom "data/cuba" \
  --rect-size "-50 50 -50 50 -50 50" \
  --origin "-55 -55 -55" \
  --spacing "1 1 1" \
  --dim "110 110 110" \
  --patient-id "TestDICOM01" \
  --patient-name "CUBA^MANIQUI" \
  --patient-pos "hfs"

# plastimatch ha creado un archivo de estructuras y uno de dosis que no queremos
!rm -f data/cuba/rtss*.dcm
!rm -f data/cuba/dose*.dcm

# Creación de la máscara para el contorno externo
!plastimatch synth \
  --background 0 \
  --foreground 1 \
  --pattern rect \
  --output "data/cuba/ss/BODY.mha" \
  --rect-size "-50 50 -50 50 -50 50" \
  --origin "-55 -55 -55" \
  --spacing "1 1 1" \
  --dim "500 500 500"

# Creación de la máscara para el PTV
!plastimatch synth \
  --background 0 \
  --foreground 1 \
  --pattern "sphere" \
  --output "data/cuba/ss/PTV.mha" \
  --radius "10" \
  --center "0 0 0" \
  --origin "-55 -55 -55" \
  --spacing "1 1 1" \
  --dim "500 500 500"

# Creación de un archivo DICOM de estructuras a partir de las máscaras manteniendo los demográficos del paciente.
!plastimatch convert \
  --input-prefix data/cuba/ss \
  --output-dicom data/cuba \
  --referenced-ct data/cuba

### Descarga

Para bajar los archivos de la nube Colab a nuestro disco local, comprimimos los datos para facilitar la descarga


In [ ]:
!zip -q -j /content/cuba.zip /content/data/cuba/*.dcm

La descarga la podemos hacer manualmente en el menú gráfico lateral de Archivos o ejecutando la celda siguiente

In [ ]:
from google.colab import files
files.download('/content/cuba.zip')

Los datos se eliminarán cuando se desconecte el entorno de ejecución. Para eliminarlos directamente, ejecutar la siguiente celda

In [ ]:
!rm -f /content/cuba.zip
!rm -rf data